# Data Mining Project

## Index
1. [Introduction](#introduction)
2. [Data Collection](#data-collection)
3. [Data Preprocessing](#data-preprocessing)
4. [Exploratory Data Analysis (EDA)](#eda)
5. [Feature Engineering](#feature-engineering)
6. [Model Selection and Training](#model-selection)
7. [Model Evaluation](#model-evaluation)
8. [Conclusion](#conclusion)  

1. <a name="introduction"></a>Introduction
   - Overview of the project
   - Objectives and goals

2. <a name="data-collection"></a>Data Collection
   - Description of data sources
   - Methods of data collection

3. <a name="data-preprocessing"></a>Data Preprocessing
   - Data cleaning techniques
   - Handling missing values
   - Data transformation and normalization

4. <a name="eda"></a>Exploratory Data Analysis (EDA)
   - Summary statistics
   - Data visualization techniques
   - Identifying patterns and trends

5. <a name="feature-engineering"></a>Feature Engineering
   - Feature selection methods
   - Creating new features
   - Dimensionality reduction techniques

6. <a name="model-selection"></a>Model Selection and Training
   - Overview of algorithms considered
   - Training procedures
   - Hyperparameter tuning

7. <a name="model-evaluation"></a>Model Evaluation
   - Evaluation metrics
   - Cross-validation results
   - Comparison of model performance

8. <a name="conclusion"></a>Conclusion
   - Summary of findings
   - Future work and improvements

In [47]:
import sqlite3
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

from itertools import product
from scipy.stats import skewnorm

from datetime import datetime
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

## DATA WRANGLING CUSTOMERS

In [48]:
customers = pd.read_csv("https://raw.githubusercontent.com/catamina07/datamining-group/main/data/DM_AIAI_CustomerDB.csv")
customers.head()

,Unnamed: 0,Loyalty#,First Name,Last Name,Customer Name,Country,Province or State,City,Latitude,Longitude,...,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType
0,0,480934,Cecilia,Householder,Cecilia Householder,Canada,Ontario,Toronto,43.653225,-79.383186,...,female,Bachelor,Urban,70146.0,Married,Star,2/15/2019,NaN,3839.14,Standard
1,1,549612,Dayle,Menez,Dayle Menez,Canada,Alberta,Edmonton,53.544388,-113.490930,...,male,College,Rural,0.0,Divorced,Star,3/9/2019,NaN,3839.61,Standard
2,2,429460,Necole,Hannon,Necole Hannon,Canada,British Columbia,Vancouver,49.282730,-123.120740,...,male,College,Urban,0.0,Single,Star,7/14/2017,1/8/2021,3839.75,Standard
3,3,608370,Queen,Hagee,Queen Hagee,Canada,Ontario,Toronto,43.653225,-79.383186,...,male,College,Suburban,0.0,Single,Star,2/17/2016,NaN,3839.75,Standard
4,4,530508,Claire,Latting,Claire Latting,Canada,Quebec,Hull,45.428730,-75.713364,...,male,Bachelor,Suburban,97832.0,Married,Star,10/25/2017,NaN,3842.79,2021 Promotion


In [49]:
print(customers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16921 entries, 0 to 16920
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               16921 non-null  int64  
 1   Loyalty#                 16921 non-null  int64  
 2   First Name               16921 non-null  object 
 3   Last Name                16921 non-null  object 
 4   Customer Name            16921 non-null  object 
 5   Country                  16921 non-null  object 
 6   Province or State        16921 non-null  object 
 7   City                     16921 non-null  object 
 8   Latitude                 16921 non-null  float64
 9   Longitude                16921 non-null  float64
 10  Postal code              16921 non-null  object 
 11  Gender                   16921 non-null  object 
 12  Education                16921 non-null  object 
 13  Location Code            16921 non-null  object 
 14  Income                

In [50]:
# Drop the first column which is an unnecessary index column
customers.drop(columns=customers.columns[0], inplace=True)

In [51]:
# Drop the columns "First Name" and "Last Name" as they are not relevant for our analysis, a
# nd they contain many unique values that are already represented in the "Customer Name" column
customers.drop(columns=["First Name", "Last Name"], inplace=True)

In [52]:
#How many countries are represented in the dataset?
print(customers['Country'].nunique())
print(customers['Country'].unique())

1
['Canada']


In [53]:
# Checking for duplicates
print(f"Number of duplicate rows: {customers.duplicated().sum()}")

Number of duplicate rows: 0


In [54]:
# Checiking if there are two or more instances with the same LoyaltyCardNumber
print(f"Number of duplicate LoyaltyCardNumber: {customers['Loyalty#'].duplicated().sum()}")


Number of duplicate LoyaltyCardNumber: 164


In [55]:
# Show all rows where Loyalty# is duplicated
duplicates = customers[customers['Loyalty#'].duplicated(keep=False)]

# Count frequency of each Loyalty#
dup_counts = customers['Loyalty#'].value_counts()
duplicates_only = dup_counts[dup_counts > 1]
print("Duplicate Loyalty numbers and counts:")
duplicates_only

print("Number of unique Loyalty numbers that are duplicated:", len(duplicates_only))
print("Total rows involved in duplicates:", duplicates_only.sum())



Duplicate Loyalty numbers and counts:
Number of unique Loyalty numbers that are duplicated: 163
Total rows involved in duplicates: 327


In [56]:
#How many unique loyalty card numbers are there?
print(customers['Loyalty#'].nunique())

16757


In [57]:
customers[customers['Loyalty#'] == 678205]

,Loyalty#,Customer Name,Country,Province or State,City,Latitude,Longitude,Postal code,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType
9546,678205,Arnold Sirles,Canada,Quebec,Hull,45.428730,-75.713364,J8Y 3Z5,male,Bachelor,Urban,48448.0,Married,Nova,9/27/2018,NaN,20230.97,Standard
9995,678205,Chanelle Essman,Canada,New Brunswick,Moncton,46.087818,-64.778229,E1A 2A7,female,College,Rural,0.0,Single,Star,9/27/2015,NaN,2180.08,Standard
16408,678205,Genna Podwoski,Canada,Ontario,London,42.984924,-81.245277,M5B 3E4,male,Bachelor,Suburban,35095.0,Divorced,Star,3/3/2021,11/3/2021,16979.80,2021 Promotion


In [58]:
# Since there are repeated Loyalty# entries, we will drop all entries with repeated Loyalty#,
# as we cannot determine which entry is the correct one to keep, and we do not want to introduce bias by
# choosing one over the other.
customers = customers[~customers['Loyalty#'].isin(duplicates_only.index)]
print(f"New number of rows after dropping duplicates: {customers.shape[0]}")

New number of rows after dropping duplicates: 16594


In [59]:
customers.isna().sum()

Loyalty#                       0
Customer Name                  0
Country                        0
Province or State              0
City                           0
Latitude                       0
Longitude                      0
Postal code                    0
Gender                         0
Education                      0
Location Code                  0
Income                        20
Marital Status                 0
LoyaltyStatus                  0
EnrollmentDateOpening          0
CancellationDate           14327
Customer Lifetime Value       20
EnrollmentType                 0
dtype: int64

In [60]:
mask = customers['Customer Lifetime Value'].isna() & customers['Income'].isna()
print(mask.sum())  # both are missing
print(customers[mask].index)  

20
Index([16901, 16902, 16903, 16904, 16905, 16906, 16907, 16908, 16909, 16910,
       16911, 16912, 16913, 16914, 16915, 16916, 16917, 16918, 16919, 16920],
      dtype='int64')


In [61]:
# Total number of rows
total_rows = len(customers)

# Columns to check with NaN values that are critical
columns_to_check = ['Income', 'Customer Lifetime Value']

# Count how many rows have NaN in any of these columns
rows_with_nulls = customers[columns_to_check].isnull().any(axis=1).sum()

# Percentage of rows that would be removed
percent_rows = rows_with_nulls / total_rows * 100
print(f"Rows to remove: {rows_with_nulls} ({percent_rows:.2f}%)")

# Drop rows only if less than 5% of data
if percent_rows < 5:
    customers = customers.dropna(subset=columns_to_check)
    print("Rows removed.")
else:
    print("Not removing rows, they represent more than 5% of total.")

Rows to remove: 20 (0.12%)
Rows removed.


In [62]:
# For the column CancelationDate we found it relevant to keep this info, and so those entries that are 
#None are replaced with "Not Cancelled"

customers['CancellationDate'].fillna('Not Canceled', inplace=True)
customers.tail()

,Loyalty#,Customer Name,Country,Province or State,City,Latitude,Longitude,Postal code,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType
16896,823768,Mandy Sammarco,Canada,British Columbia,Vancouver,49.282730,-123.120740,V6E 3Z3,female,College,Rural,0.0,Married,Star,12/22/2015,Not Canceled,61850.19,Standard
16897,680886,Jamee Ahlm,Canada,Saskatchewan,Regina,50.445210,-104.618900,S1J 3C5,female,Bachelor,Rural,78310.0,Married,Star,9/6/2017,Not Canceled,67907.27,Standard
16898,776187,Janina Lumb,Canada,British Columbia,Vancouver,49.282730,-123.120740,V5R 1W3,male,College,Urban,0.0,Single,Star,3/24/2017,Not Canceled,74228.52,Standard
16899,615459,Dannie Paplow,Canada,Quebec,Montreal,45.501690,-73.567253,H2Y 4R4,male,Bachelor,Urban,58958.0,Married,Star,4/22/2021,12/22/2021,83325.38,2021 Promotion
16900,652627,Ariane Peyton,Canada,Manitoba,Winnipeg,49.895138,-97.138374,R2C 0M5,female,Bachelor,Suburban,58958.0,Married,Star,12/3/2018,8/3/2019,83325.38,Standard


In [ ]:
# We will also add a new column indicating whether a customer is canceled or not, where 0 means not canceled and 1 means canceled

customers['IsCanceled'] = np.where(customers['CancellationDate'] == 'Not Canceled', 0, 1)
customers.head()



,Loyalty#,Customer Name,Country,Province or State,City,Latitude,Longitude,Postal code,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType,IsCanceled
0,480934,Cecilia Householder,Canada,Ontario,Toronto,43.653225,-79.383186,M2Z 4K1,female,Bachelor,Urban,70146.0,Married,Star,2/15/2019,Not Canceled,3839.14,Standard,0
1,549612,Dayle Menez,Canada,Alberta,Edmonton,53.544388,-113.490930,T3G 6Y6,male,College,Rural,0.0,Divorced,Star,3/9/2019,Not Canceled,3839.61,Standard,0
2,429460,Necole Hannon,Canada,British Columbia,Vancouver,49.282730,-123.120740,V6E 3D9,male,College,Urban,0.0,Single,Star,7/14/2017,1/8/2021,3839.75,Standard,1
3,608370,Queen Hagee,Canada,Ontario,Toronto,43.653225,-79.383186,P1W 1K4,male,College,Suburban,0.0,Single,Star,2/17/2016,Not Canceled,3839.75,Standard,0
4,530508,Claire Latting,Canada,Quebec,Hull,45.428730,-75.713364,J8Y 3Z5,male,Bachelor,Suburban,97832.0,Married,Star,10/25/2017,Not Canceled,3842.79,2021 Promotion,0


## DATA WRANGLING FLIGHTS

In [63]:
flights = pd.read_csv("https://raw.githubusercontent.com/catamina07/datamining-group/main/data/DM_AIAI_FlightsDB.csv")
flights.head()

,Loyalty#,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed
0,413052,2021,12,12/1/2021,2.0,2.0,9384.0,938.0,0.0,0.0
1,464105,2021,12,12/1/2021,0.0,0.0,0.0,0.0,0.0,0.0
2,681785,2021,12,12/1/2021,10.0,3.0,14745.0,1474.0,0.0,0.0
3,185013,2021,12,12/1/2021,16.0,4.0,26311.0,2631.0,3213.0,32.0
4,216596,2021,12,12/1/2021,9.0,0.0,19275.0,1927.0,0.0,0.0
